reworking of this:

[http://localhost:8888/notebooks/training-data-analyst/courses/machine_learning/deepdive/03_tensorflow/e_cloudmle.ipynb#Deploy-model]

In [15]:
import tensorflow as tf
import pandas as pd
import numpy as np
import shutil
import os
print tf.__version__

1.12.0


In [16]:
PROJECT = 'kouzoh-p-michal'   
BUCKET = 'ml-training-kouzoh-p-michal-appspot-com' 
REGION = 'us-central1' 

MODEL_NAME = 'taxifare'
MODEL_VERSION = 'v1'
TRAINING_DIR = './data/taxi_trained'

In [17]:
# for bash
os.environ['PROJECT'] = PROJECT
os.environ['BUCKET'] = BUCKET
os.environ['REGION'] = REGION
os.environ['TFVERSION'] = '1.8' 
os.environ['CLOUDSDK_PYTHON'] = 'python2'
os.environ['MODEL_NAME'] = MODEL_NAME
os.environ['MODEL_VERSION'] = MODEL_VERSION
os.environ['TRAINING_DIR'] = TRAINING_DIR 

## Training data check
Data is prepared as CSVs and uploaded to the bucket in a separate notebook

In [18]:
%%bash
#gsutil cp gs://$BUCKET/taxifare/ch4/taxi_preproc/test.csv-00000-of-00001 ./data/test.csv
#gsutil cp gs://$BUCKET/taxifare/ch4/taxi_preproc/train.csv-00000-of-00001 ./data/train.csv
#gsutil cp gs://$BUCKET/taxifare/ch4/taxi_preproc/valid.csv-00000-of-00001 ./data/valid.csv

## Remove trained model data

In [19]:
%%bash
#rm -rf $TRAINING_DIR/*

## Train model locally

In [10]:
%%bash
# Setup python so it sees the task module which controls the model.py
export PYTHONPATH=${PYTHONPATH}:${PWD}/${MODEL_NAME}
python -m trainer.task \
   --train_data_paths=./data/train.csv \
   --eval_data_paths=./data/valid.csv  \
   --output_dir=${TRAINING_DIR} \
   --train_steps=1000 --job-dir=./data/tmp

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_global_id_in_cluster': 0, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f703ea6b290>, '_model_dir': './data/taxi_trained', '_protocol': None, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_device_fn': None, '_experimental_distribute': None, '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_evaluation_master': '', '_eval_distribute': None, '_train_distribute': None, '_master': ''}
INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorfl

## Train model locally using gcloud

In [13]:
%%bash
# Use Cloud Machine Learning Engine to train the model in local file system
gcloud ml-engine local train \
   --module-name=trainer.task \
   --package-path=${PWD}/${MODEL_NAME}/trainer \
   -- \
   --train_data_paths=${PWD}/taxi-train.csv \
   --eval_data_paths=${PWD}/taxi-valid.csv  \
   --train_steps=1000 \
   --output_dir=${PWD}/${TRAINING_DIR} 

INFO:tensorflow:TF_CONFIG environment variable: {u'environment': u'cloud', u'cluster': {}, u'job': {u'args': [u'--train_data_paths=/var/dev/ml-experiments/taxifares/taxi-train.csv', u'--eval_data_paths=/var/dev/ml-experiments/taxifares/taxi-valid.csv', u'--train_steps=1000', u'--output_dir=/var/dev/ml-experiments/taxifares/./data/taxi_trained'], u'job_name': u'trainer.task'}, u'task': {}}
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_global_id_in_cluster': 0, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f653776f990>, '_model_dir': '/var/dev/ml-experiments/taxifares/./data/taxi_trained', '_protocol': None, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iteration

## Submit trainig job to cloud

In [20]:
%%bash
OUTDIR=gs://${BUCKET}/taxifare/ch4/trained
JOBNAME=${MODEL_NAME}_$(date -u +%y%m%d_%H%M%S)
echo $OUTDIR $REGION $JOBNAME
# Clear the Cloud Storage Bucket used for the training job
gsutil -m rm -rf $OUTDIR
gcloud ml-engine jobs submit training $JOBNAME \
   --region=$REGION \
   --module-name=trainer.task \
   --package-path=${PWD}/${MODEL_NAME}/trainer \
   --job-dir=$OUTDIR \
   --staging-bucket=gs://$BUCKET \
   --scale-tier=BASIC \
   --runtime-version=$TFVERSION \
   -- \
   --train_data_paths="gs://${BUCKET}/taxifare/ch4/taxi_preproc/train*" \
   --eval_data_paths="gs://${BUCKET}/taxifare/ch4/taxi_preproc/valid*"  \
   --output_dir=$OUTDIR \
   --train_steps=10000

gs://ml-training-kouzoh-p-michal-appspot-com/taxifare/ch4/trained us-central1 taxifare_190215_152520
jobId: taxifare_190215_152520
state: QUEUED


CommandException: 1 files/objects could not be removed.
Job [taxifare_190215_152520] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ml-engine jobs describe taxifare_190215_152520

or continue streaming the logs with the command

  $ gcloud ml-engine jobs stream-logs taxifare_190215_152520


Check what's in the directory where cloud training outputs the model

In [24]:
%%bash
gsutil ls gs://${BUCKET}/taxifare/ch4/trained/export/exporter

gs://ml-training-kouzoh-p-michal-appspot-com/taxifare/ch4/trained/export/exporter/
gs://ml-training-kouzoh-p-michal-appspot-com/taxifare/ch4/trained/export/exporter/1550244453/
gs://ml-training-kouzoh-p-michal-appspot-com/taxifare/ch4/trained/export/exporter/1550244989/


Deploy model - this just creates a model without any content

In [25]:
%%bash
gcloud ml-engine models create ${MODEL_NAME} --regions $REGION

Created ml engine model [projects/kouzoh-p-michal/models/taxifare].


 Deploy trained version to the model we created


In [26]:
%%bash
MODEL_LOCATION=$(gsutil ls gs://${BUCKET}/taxifare/ch4/trained/export/exporter | tail -1)

echo "MODEL_LOCATION = ${MODEL_LOCATION}"

gcloud ml-engine versions create ${MODEL_VERSION} --model ${MODEL_NAME} --origin ${MODEL_LOCATION} --runtime-version $TFVERSION

MODEL_LOCATION = gs://ml-training-kouzoh-p-michal-appspot-com/taxifare/ch4/trained/export/exporter/1550244989/


Creating version (this might take a few minutes)......
..............................................................................................................................................................................................................................................................................................................................done.
